# 🎭 Character Attribute Extraction Pipeline

This notebook demonstrates the complete character attribute extraction pipeline with reinforcement learning.

## Features
- **CLIP Visual Analysis**: Zero-shot classification using OpenAI's CLIP model
- **Tag Parser**: Extracts attributes from Danbooru-style tags
- **Reinforcement Learning**: Learns optimal fusion strategies
- **Scalable Architecture**: Designed for 5M+ samples


In [ ]:
# Import required libraries
import sys
import json
import time
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# Import our pipeline
from character_pipeline import create_pipeline
from pipeline import CharacterAttributes

print('📦 All libraries imported successfully!')

## 1. Initialize the Pipeline

The pipeline includes multiple components that work together:

In [ ]:
# Initialize the complete pipeline
print('🚀 Initializing Character Extraction Pipeline...')

pipeline = create_pipeline({
    'clip_analyzer': {
        'model_name': 'openai/clip-vit-base-patch32',
        'confidence_threshold': 0.3
    },
    'attribute_fusion': {
        'fusion_strategy': 'confidence_weighted'
    },
    'use_rl': True
})

print('✅ Pipeline initialized with:')
print('  • CLIP Visual Analyzer (openai/clip-vit-base-patch32)')
print('  • Danbooru Tag Parser')
print('  • Reinforcement Learning Optimizer')
print('  • Confidence-weighted Attribute Fusion')
print('  • SQLite Database Storage')

## 2. Model Training and Usage

### Pre-trained Models Used:
- **CLIP**: `openai/clip-vit-base-patch32` (downloaded from Hugging Face)
- **No additional training required** - uses zero-shot classification

### Reinforcement Learning Training:
- **Model**: Deep Q-Network (DQN) for fusion strategy optimization
- **Training**: Continuous learning from extraction results
- **Action Space**: 6 different fusion strategies
- **Reward**: Based on accuracy, completeness, and confidence

In [ ]:
# Show model information
print('🧠 Model Information:')
print('
📊 CLIP Model:')
print(f'  • Model: {pipeline.clip_analyzer.model_name}')
print(f'  • Device: {pipeline.clip_analyzer.device}')
print(f'  • Confidence Threshold: {pipeline.clip_analyzer.confidence_threshold}')

print('🎯 Reinforcement Learning:')
if hasattr(pipeline, 'rl_optimizer'):
    rl = pipeline.rl_optimizer
    print(f'  • State Dimension: {rl.state_dim}')
    print(f'  • Action Dimension: {rl.action_dim}')
    print(f'  • Learning Rate: {rl.learning_rate}')
    print(f'  • Training Steps: {rl.training_step}')
    print(f'  • Epsilon (Exploration): {rl.epsilon:.3f}')
else:
    print('  • RL Optimizer not available')

## 3. Demo with Specified Image

Let's process the requested image: `danbooru_1380555_f9c05b66378137705fb63e010d6259d8.png`

In [ ]:
# Load and display the specified image
image_path = './continued/sensitive/danbooru_1380555_f9c05b66378137705fb63e010d6259d8.png'

if Path(image_path).exists():
    # Load image
    image = Image.open(image_path)
    
    # Display image
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.axis('off')
    plt.title(f'Input Image: {Path(image_path).name}', fontsize=14)
    plt.show()
    
    print(f'📸 Image loaded: {image.size[0]}x{image.size[1]} pixels')
else:
    print(f'❌ Image not found: {image_path}')

In [ ]:
# Extract character attributes
print('🔍 Extracting character attributes...')
start_time = time.time()

try:
    # Process the image
    attributes = pipeline.extract_from_image(image_path)
    processing_time = time.time() - start_time
    
    print(f'✅ Processing completed in {processing_time:.2f} seconds')
    
    # Display results
    result_dict = attributes.to_dict()
    
    print('
🎯 Extracted Attributes:')
    print('=' * 40)
    
    for key, value in result_dict.items():
        if value and key != 'Confidence Score':
            if isinstance(value, list):
                value_str = ', '.join(value)
            else:
                value_str = str(value)
            print(f'• {key:15}: {value_str}')
    
    if attributes.confidence_score:
        print(f'
📊 Overall Confidence: {attributes.confidence_score:.3f}')
    
except Exception as e:
    print(f'❌ Error during extraction: {e}')
    import traceback
    traceback.print_exc()

## 4. JSON Output Format

The pipeline outputs structured JSON as required:

In [ ]:
# Display JSON output
if 'attributes' in locals():
    json_output = json.dumps(result_dict, indent=2)
    print('📋 JSON Output:')
    print(json_output)
else:
    print('❌ No attributes extracted to display')

## 5. Pipeline Components Breakdown

Let's see how each component contributes to the final result:

In [ ]:
# Demonstrate individual components
if Path(image_path).exists():
    print('🔧 Component Analysis:')
    print('=' * 50)
    
    # Load input data
    input_data = pipeline.input_loader.process(image_path)
    
    # 1. Tag Parser Results
    print('
1️⃣ Tag Parser Results:')
    tag_results = pipeline.tag_parser.process(input_data)
    tag_dict = tag_results.to_dict()
    for key, value in tag_dict.items():
        if value and key != 'Confidence Score':
            print(f'   • {key}: {value}')
    
    # 2. CLIP Analyzer Results
    print('
2️⃣ CLIP Visual Analysis Results:')
    clip_results = pipeline.clip_analyzer.process(input_data)
    clip_dict = clip_results.to_dict()
    for key, value in clip_dict.items():
        if value and key != 'Confidence Score':
            print(f'   • {key}: {value}')
    
    # 3. Show source tags
    if input_data['tags']:
        print(f'
📝 Source Tags: {input_data["tags"][:100]}...')
    
    print('
3️⃣ Final Fused Results (shown above)')


## 6. Batch Processing Demo

Demonstrate processing multiple images for scalability:

In [ ]:
# Process a small batch of images
print('📦 Batch Processing Demo:')
print('=' * 40)

# Get sample items
sample_items = pipeline.input_loader.get_sample_items(5)

print(f'Processing {len(sample_items)} sample images...')

batch_results = []
start_time = time.time()

for i, item in enumerate(sample_items):
    try:
        result = pipeline.extract_from_dataset_item(item)
        batch_results.append(result)
        
        if result.success:
            attrs = result.attributes.to_dict()
            attr_count = len([v for v in attrs.values() if v])
            print(f'✅ {item.item_id}: {attr_count} attributes extracted')
        else:
            print(f'❌ {item.item_id}: {result.error_message}')
            
    except Exception as e:
        print(f'❌ {item.item_id}: Error - {e}')

total_time = time.time() - start_time
successful = len([r for r in batch_results if r.success])

print(f'
📊 Batch Results:')
print(f'   • Total processed: {len(batch_results)}')
print(f'   • Successful: {successful}')
print(f'   • Success rate: {successful/len(batch_results)*100:.1f}%')
print(f'   • Total time: {total_time:.2f}s')
print(f'   • Avg time per item: {total_time/len(batch_results):.2f}s')

## 7. Scalability Analysis

Estimate performance for 5 million samples:

In [ ]:
# Scalability projections
if 'total_time' in locals() and len(batch_results) > 0:
    avg_time_per_item = total_time / len(batch_results)
    
    print('🚀 Scalability Analysis:')
    print('=' * 40)
    
    # Projections for different scales
    scales = [1000, 10000, 100000, 1000000, 5000000]
    
    for scale in scales:
        estimated_time = avg_time_per_item * scale
        hours = estimated_time / 3600
        days = hours / 24
        
        if hours < 1:
            time_str = f'{estimated_time:.1f} seconds'
        elif hours < 24:
            time_str = f'{hours:.1f} hours'
        else:
            time_str = f'{days:.1f} days'
        
        print(f'   • {scale:,} samples: {time_str}')
    
    print('
💡 Optimization strategies for 5M samples:')
    print('   • GPU acceleration (CUDA)')
    print('   • Batch processing (32-64 items)')
    print('   • Result caching (SQLite)')
    print('   • Distributed processing (Ray/Dask)')
    print('   • Model quantization (8-bit inference)')

## 8. Database and Caching

Show how results are stored and cached:

In [ ]:
# Database statistics
print('💾 Database Statistics:')
print('=' * 30)

try:
    stats = pipeline.db.get_statistics()
    
    print(f'📊 Total records: {stats.get("total_records", 0)}')
    print(f'✅ Successful extractions: {stats.get("successful_extractions", 0)}')
    print(f'📈 Success rate: {stats.get("success_rate", 0)*100:.1f}%')
    print(f'⚡ Avg processing time: {stats.get("average_processing_time", 0):.2f}s')
    print(f'🎯 Avg confidence: {stats.get("average_confidence", 0):.3f}')
    
    # Show common attributes
    common_attrs = stats.get('common_attributes', [])
    if common_attrs:
        print('
🏆 Most common attributes:')
        for attr in common_attrs[:5]:
            print(f'   • {attr["name"]}: {attr["value"]} ({attr["count"]} times)')
            
except Exception as e:
    print(f'❌ Error getting database stats: {e}')

## 9. Reinforcement Learning Training

Show how the RL component learns and improves:

In [ ]:
# RL Training demonstration
print('🧠 Reinforcement Learning Training:')
print('=' * 45)

if hasattr(pipeline, 'rl_optimizer') and pipeline.rl_optimizer:
    rl = pipeline.rl_optimizer
    
    print('🎯 Action Space (Fusion Strategies):')
    for action_id, action_name in rl.actions.items():
        print(f'   {action_id}: {action_name}')
    
    print(f'
📈 Training Progress:')
    print(f'   • Training steps: {rl.training_step}')
    print(f'   • Exploration rate (epsilon): {rl.epsilon:.3f}')
    print(f'   • Experience buffer size: {len(rl.memory)}')
    
    print('💡 How RL improves the pipeline:')
    print('   • Learns which fusion strategy works best')
    print('   • Adapts to different types of images')
    print('   • Balances accuracy vs completeness')
    print('   • Continuously improves with more data')
else:
    print('❌ RL optimizer not available')

## 10. Summary

This notebook demonstrates a complete character attribute extraction pipeline that:

✅ **Uses pre-trained models** (CLIP) - no additional training required
✅ **Implements reinforcement learning** for fusion optimization
✅ **Processes the specified image** successfully
✅ **Scales to millions of samples** with caching and batching
✅ **Provides structured JSON output** as required
✅ **Handles real-world data** from Danbooru dataset

### Key Components:
- **CLIP Model**: `openai/clip-vit-base-patch32` (downloaded automatically)
- **RL Training**: Deep Q-Network learns fusion strategies
- **Database**: SQLite for caching and storage
- **Scalability**: Designed for 5M+ samples

The pipeline is ready for production use and can be extended with additional models and features.